# RAG - How to query

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

weaviate_url = os.getenv("WEAVIATE_URL")
openai_key = os.getenv("OPENAI_API_KEY")
openai_url = os.getenv("OPENAI_URL")

print(weaviate_url, openai_key, openai_url)

## Connect to Weaviate

In [ ]:
import weaviate

client = weaviate.connect_to_local(
    host=weaviate_url,
    headers={
        "X-OpenAI-Api-Key": openai_key,
    }
)

client.is_ready()

### Start with (R) - Retrieval

In [6]:
wiki = client.collections.get("Wiki")

response = wiki.query.near_text(
    query="How do planes fly",
    limit=5,
    return_properties=["text", "title"]
)

for item in response.objects:
    print(item.properties)

{'text': 'There are a number of ways of explaining the production of lift; some are more complicated than others, some have been shown to be false. The simplest explanation is that the wing deflects air downward, and the reaction pushes the wing up.', 'title': 'Lift (force)'}
{'text': 'At very high speeds, objects that are inside the atmosphere become very hot. This can cause the previously mentioned boundary layer of the plane to expand. When the boundary layer expands, the aircraft gets even hotter. Put simply, planes moving extremely fast experience lots and lots of drag and get very very hot. This process is caused by the viscosity of air, which is why this is called "viscous interaction."', 'title': 'Hypersonic speed'}
{'text': 'Anderson, David F. and Scott Eberhardt. Understanding Flight. Chicago: McGraw Hill Professional, 2009. .', 'title': 'Boeing 737'}
{'text': 'A paper plane is a toy airplane made from paper. The paper is folded in a certain way so that the folded paper actua

### Add (AG) - augmented generation - to make full RAG

#### Single Prompt

> Generate a response per **retrieved** object.

In [7]:
wiki = client.collections.get("Wiki")

response = wiki.generate.near_text(
    query="How do planes fly",
    # auto_limit=1,
    limit=5,
    single_prompt="Explain what this is about? {text}"
)

print("=== Source ===")
for item in response.objects:
    print(item.properties)
    print(item.generated)

=== Source ===
{'text': 'There are a number of ways of explaining the production of lift; some are more complicated than others, some have been shown to be false. The simplest explanation is that the wing deflects air downward, and the reaction pushes the wing up.', 'title': 'Lift (force)', 'wiki_id': '20231101.simple_45913_3', 'url': 'https://simple.wikipedia.org/wiki/Lift%20%28force%29'}
This passage is about the physics of how an airplane wing, or airfoil, generates lift, which is the force that allows the plane to rise and stay in the air. The simplest explanation given is based on Newton's third law of motion: for every action, there is an equal and opposite reaction. So
{'text': 'At very high speeds, objects that are inside the atmosphere become very hot. This can cause the previously mentioned boundary layer of the plane to expand. When the boundary layer expands, the aircraft gets even hotter. Put simply, planes moving extremely fast experience lots and lots of drag and get ver

#### Grouped Task

> Generate one response based on all **retrieved** objects.

In [ ]:
wiki = client.collections.get("Wiki")

response = wiki.generate.near_text(
    query="How do planes fly",
    # auto_limit=1,
    limit=5,
    grouped_task="Explain, how do planes fly? Please only use the provided content."
)

print("=== Generated Response ===")
print(response.generated)

print("=== Source ===")
for item in response.objects:
    print(item.properties)

#### Specify which properties to use for grouped task

In [8]:
wiki = client.collections.get("Wiki")

response = wiki.generate.near_text(
    query="How do planes fly",
    auto_limit=1,
    grouped_task="Explain, how do planes fly? Please only use the provided content.",
    grouped_properties=["text", "title"]
)

print("=== Generated Response ===")
print(response.generated)

print("=== Source ===")
for item in response.objects:
    print(item.properties)

=== Generated Response ===
Planes fly primarily due to a force called lift. The simplest explanation of lift is that the wing of the plane deflects air downward, and the reaction pushes the wing up, allowing the plane to ascend. This process is more complex at very high speeds. When a plane is moving extremely fast, it experiences a lot of drag and gets very hot. This is due to a process called "viscous interaction," caused by the viscosity of air. The boundary layer of the plane expands due to the heat, making the aircraft even hotter. This principle of flight was initially tested using paper planes, which are folded in a specific way to glide through the air. Today, the making of paper planes is a hobby and a craft, but the principles behind their flight are the same as those that allow real planes to fly.
=== Source ===
{'text': 'There are a number of ways of explaining the production of lift; some are more complicated than others, some have been shown to be false. The simplest expl

## Close the client

In [ ]:
client.close()